In [393]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [394]:
def annotateText(text):
  # request = "http://api.dbpedia-spotlight.org/en/annotate?confidence=0.5&text="+text
  request = "http://141.57.9.112:2222/rest/annotate?confidence=0.5&text="+text
  response = requests.get(request)
  page_content = BeautifulSoup(response.content,"html.parser")
  return page_content.find_all('a')

In [395]:
def findResource(resource):
  query = "select ?label where {<" + resource + "> rdfs:label ?label. filter(lang(?label) = 'es')}"
  request = "http://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query="+query+"&format=application%2Fsparql-results%2Bjson&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+"
  response = requests.get(request).json()
  try:
    out = response["results"]["bindings"][0]["label"]["value"]
  except IndexError:
    out = None
  return out

In [396]:
def getResourceAndEntity(resourceList):
  dict = {}
  for item in resourceList:
    if item==None:
      continue
    dict[item.text] = item.get('href')
  return dict

In [397]:
def getReplacedText(input_text,output_text,dict):

  input_text = input_text.lower()
  output_text = output_text.lower()

  if bool(dict)==False:
    return input_text,output_text

  count = 0

  for entity,resource in dict.items():
    input_text = input_text.replace(entity.lower(),"entity")
    
    if entity in output_text:
      output_text = output_text.replace(entity.lower(),"entidad")
    else:
      entity = findResource(resource)
      if entity!=None:
        output_text = output_text.replace(entity.lower(),"entidad")
    count+=1
  return input_text,output_text

In [398]:
# df=pd.read_csv('/content/drive/My Drive/data/entity_sets/en-es-train.csv')
# df=pd.read_csv('/content/drive/My Drive/data/entity_sets/en-es-test.csv')

# df=df.dropna(subset=['input_lang','output_lang'])

df_1=pd.read_csv('/content/drive/My Drive/data/entity_sets/en-es-train.csv')
df_2=pd.read_csv('/content/drive/My Drive/data/entity_sets/en-es-test.csv')
frames = [df_1,df_2]
df = pd.concat(frames)

In [399]:
df = df.dropna(subset=['input_lang','output_lang'])
df = df.drop_duplicates(subset=['input_lang','output_lang'], keep='first', inplace=False)

In [400]:
df_new = df[df.correctly_annotated==1]
df_new = df_new.reset_index(drop=True)

In [401]:
df.shape

(606, 3)

In [402]:
df_new.head(20)

,input_lang,output_lang,correctly_annotated
0,Which river does the Brooklyn Bridge cross?,¿Por qué río cruza la Brooklyn Bridge?,1.0
1,Give me the websites of companies with more th...,Dame todas las páginas de Web de empresas con ...,1.0
2,What is the official website of Tom Cruise?,¿Cuál el la página de Web oficial de Tom Cruise?,1.0
3,Give me all movies with Tom Cruise.,Dame todas las películas con Tom Cruise,1.0
4,Who created Wikipedia?,¿Quién creó Wikipedia?,1.0
5,How many films did Hal Roach produce?,¿Cuántas películas ha producido Hal Roach?,1.0
6,In which country does the Nile start?,¿En qué país nace el Nilo?,1.0
7,How tall is Claudia Schiffer?,¿Cómo de alta es Claudia Schiffer?,1.0
8,Which television shows were created by Walt Di...,¿Qué series televisivas ideó Walt Disney?,1.0
9,Give me all cities in New Jersey with more tha...,Dame todas las cuidades en New Jersey que teng...,1.0


In [403]:
input_list = []
output_list = []
for _, row in df_new.iterrows():
  list = annotateText(row['input_lang'])
  dict = getResourceAndEntity(list)
  input_text,output_text = getReplacedText(row['input_lang'],row['output_lang'],dict)
  input_list.append(input_text)
  output_list.append(output_text)

In [404]:
new_df = pd.DataFrame({"input_lang":input_list,"output_lang":output_list})

In [405]:
new_df.head(20)

,input_lang,output_lang
0,which river does the entity cross?,¿por qué río cruza la brooklyn bridge?
1,give me the websites of companies with more th...,dame todas las páginas de web de empresas con ...
2,what is the official website of entity?,¿cuál el la página de web oficial de entidad?
3,give me all movies with entity.,dame todas las películas con entidad
4,who created entity?,¿quién creó entidad?
5,how many films did entity produce?,¿cuántas películas ha producido entidad?
6,in which country does the entity start?,¿en qué país nace el entidad?
7,how tall is entity?,¿cómo de alta es entidad?
8,which television shows were created by entity?,¿qué series televisivas ideó entidad?
9,give me all cities in entity with more than 10...,dame todas las cuidades en new jersey que teng...


In [406]:
# new_df.to_csv('/content/drive/My Drive/data/template/en-es-test-template.csv', index=False)
# new_df.to_csv('/content/drive/My Drive/data/template/en-es-train-template.csv', index=False)
new_df.to_csv('/content/drive/My Drive/data/template/en-es-template.csv', index=False)